In [1]:
from controllables.energyplus import System

sys = System(
    building='model_with_photovoltaic.idf',
    weather='SGP_Singapore_486980_IWEC.epw',
    # TODO
    report='tmp/',
    repeat=True,
)
# sys.add('logging:progress').start()

In [2]:
sys.variables.available_keys()

calendar
Loading ITables v2.0.1 from the internet... (need help?)


In [2]:
from controllables.energyplus import Actuator, OutputVariable, OutputMeter
from controllables.core.tools.records import VariableRecords
from ray.rllib.algorithms.callbacks import DefaultCallbacks
from controllables.core import BaseVariable
from controllables.core.tools.records import VariableRecords
import numpy as _numpy_
import pythermalcomfort as pytc

class PMVVariable(BaseVariable):
    def __init__(
        self, 
        tdb: BaseVariable,
        tr: BaseVariable,
        rh: BaseVariable,
        metab_rate=1.5, clothing=.5, pmv_limit=.5,
    ):
        self.tdb = tdb
        self.tr = tr
        self.rh = rh
        self._metab_rate = _numpy_.asarray(metab_rate)
        self._clothing = _numpy_.asarray(clothing)
        self._pmv_limit = _numpy_.asarray(pmv_limit)
    
    @property
    def value(self):
        res = pytc.models.pmv_ppd(
            tdb=self.tdb.value, 
            tr=self.tr.value, 
            # calculate relative air speed
            vr=pytc.utilities.v_relative(v=0.1, met=self._metab_rate), 
            rh=self.rh.value, 
            met=self._metab_rate, 
            # calculate dynamic clothing
            clo=pytc.utilities.clo_dynamic(clo=self._clothing, met=self._metab_rate),
            limit_inputs=False,
        )['pmv']
        return res
sys.add('logging:progress').start()
tdb = sys[OutputVariable.Ref('Zone Mean Air Temperature', '1FFIRSTFLOORWEST:OPENOFFICE')]
tr = sys[OutputVariable.Ref('Zone Mean Radiant Temperature', '1FFIRSTFLOORWEST:OPENOFFICE')]
rh = sys[OutputVariable.Ref('Zone Air Relative Humidity', '1FFIRSTFLOORWEST:OPENOFFICE')]

records = VariableRecords({
    '🕰️': sys['wallclock:calendar'],
    'HVAC': 
        sys[OutputMeter.Ref('Electricity:HVAC')] ,
    'SOLAR COLLECTOR 1':sys[OutputVariable.Ref('Generator Produced DC Electricity Energy', 'SOLAR COLLECTOR 1')],
    'produce':sys[OutputMeter.Ref('Photovoltaic:ElectricityProduced')]/4,
    'AHU COOLING COIL': sys[OutputVariable.Ref('Cooling Coil Total Cooling Rate', 'AIR LOOP AHU COOLING COIL')],
    'occupancy':sys[OutputVariable.Ref('Schedule Value', 'Office_OpenOff_Occ')],
    'pmv':PMVVariable(tdb=tdb, tr=tr, rh=rh)
}).watch(sys.events['timestep'])

E0000 00:00:1731129686.279742  513517 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1731129686.283173  513517 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


  0%|          | 0/100 [00:00<?, ?it/s]

In [ ]:
import pandas as pd
import numpy as np
sys['wallclock:calendar'].value
curtime = sys['wallclock:calendar'].value
curtime = pd.to_datetime(curtime)
print(curtime)

2002-07-16 17:23:20+00:00


In [ ]:
records.dataframe().to_csv('tmp/records_baseline.csv', index=False)

In [ ]:
records.plot.scatter(x='🕰️', y='pmv').watch(records.events['change'] % 1_000)

In [ ]:
records.plot.scatter(x='🕰️', y='AHU COOLING COIL').watch(records.events['change'] % 1_000)

In [ ]:
records.plot.scatter(x='🕰️', y='HVAC').watch(records.events['change'] % 1_000)

In [ ]:
records.plot.scatter(x='🕰️', y='SOLAR COLLECTOR 1').watch(records.events['change'] % 1_000)

In [ ]:
records.plot.scatter(x='🕰️', y='produce').watch(records.events['change'] % 1_000)

In [7]:
current_time = sys['wallclock:calendar'].value


In [ ]:
(df_baseline['time'] - current_time).abs().min() = 

Timedelta('0 days 00:00:00')

In [41]:
float(df_baseline[df_baseline['time'] == current_time]['elec'])


/tmp/ipykernel_513517/3921852107.py:1: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  float(df_baseline[df_baseline['time'] == current_time]['elec'])


8720597.491576927

In [ ]:
import pandas as pd
import numpy as np
df_baseline = pd.read_csv("datasave/records_-1FWEST.csv")

if not np.issubdtype(df_baseline['time'].dtype, np.datetime64):
    df_baseline['time'] = pd.to_datetime(df_baseline['time'])

print(df_baseline['time'])

current_time = sys['wallclock:calendar'].value

# print(current_time)
if current_time in df_baseline['time']:
    print("found")
    print(df_baseline.loc[current_time])
    



0      2002-07-01 00:20:00+00:00
1      2002-07-01 00:30:00+00:00
2      2002-07-01 00:40:00+00:00
3      2002-07-01 00:50:00+00:00
4      2002-07-01 01:00:00+00:00
                  ...           
4603   2002-08-01 23:30:00+00:00
4604   2002-08-01 23:40:00+00:00
4605   2002-08-01 23:50:00+00:00
4606   2002-08-02 00:00:00+00:00
4607                         NaT
Name: time, Length: 4608, dtype: datetime64[ns, UTC]


TemporaryUnavailableError: WallClock(WallClock.Ref(calendar=True))

In [20]:
s = pd.Series(range(-3, 4))

In [37]:
s[s > 0 ]


4    1
5    2
6    3
dtype: int64

In [24]:
s

0   -3
1   -2
2   -1
3    0
4    1
5    2
6    3
dtype: int64

In [28]:
np.array([1, 2, 3]) - 10

array([-9, -8, -7])

In [34]:
bool(np.array([True, False, True]))

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()